# SQLAlchemy

In [1]:
import pandas as pd
from datetime import datetime

In [2]:
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, Integer, String, Date, Float

In [3]:
engine = sqlalchemy.create_engine('sqlite:///:memory:', echo=True)  #arad_db.db

In [4]:
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
Base = declarative_base()

In [6]:
url = 'https://www.cnb.cz/cnb/STAT.ARADY_PKG.VYSTUP?p_period=1&p_sort=1&p_des=50&p_sestuid=1538&p_uka=1%2C2%2C3&p_strid=AAD&p_od=200512&p_do=202012&p_lang=CS&p_format=2&p_decsep=.'

In [7]:
df = pd.read_csv(url, sep='|',skiprows=0,verbose=False,encoding ='utf-8',header=0)
df.iloc[:,0] = pd.to_datetime(df.iloc[:,0])

In [8]:
df.head()

,Období,UK1 Na spotřebu,UK2 Na bydlení,UK3 Ostatní
0,2005-12-31,182036.4,282662.2,58149.7
1,2006-01-31,181932.9,287555.3,58207.3
2,2006-02-28,182050.8,292363.9,58634.9
3,2006-03-31,183489.5,298946.3,59563.1
4,2006-04-30,184419.0,304568.2,60349.1


In [9]:
df.dropna(axis=0,inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169 entries, 0 to 168
Data columns (total 4 columns):
Období             169 non-null datetime64[ns]
UK1 Na spotřebu    169 non-null float64
UK2 Na bydlení     169 non-null float64
UK3 Ostatní        169 non-null float64
dtypes: datetime64[ns](1), float64(3)
memory usage: 6.6 KB


### Create table

In [6]:
Base.metadata.create_all(bind=engine)

2020-03-01 19:30:28,989 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-03-01 19:30:28,990 INFO sqlalchemy.engine.base.Engine ()
2020-03-01 19:30:28,993 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-03-01 19:30:28,994 INFO sqlalchemy.engine.base.Engine ()


In [7]:
class Loans(Base):
    __tablename__ = 'loans'
    id = Column(Integer, primary_key=True)
    #datum = Column(Date)
    spotreba = Column(Integer) # Float
    bydleni = Column(Integer)
    ostatni = Column(Integer)
    
    def __repr__(self):
        return "<Loans(Datum='', spotreba='%s, bydleni='%s, ostatni='%s')>" % (self.spotreba, self.bydleni, self.ostatni)

In [9]:
print(Loans)

<class '__main__.Loans'>


### Add data

In [20]:
session.rollback()

/Users/OndrejMartinek/opt/anaconda3/lib/python3.7/site-packages/sqlalchemy/orm/session.py:552: SAWarning: Session's state has been changed on a non-active transaction - this state will be discarded.
  "Session's state has been changed on "


In [10]:
session.commit()

2020-03-01 19:30:44,093 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-03-01 19:30:44,097 INFO sqlalchemy.engine.base.Engine INSERT INTO loans (spotreba, bydleni, ostatni) VALUES (?, ?, ?)
2020-03-01 19:30:44,099 INFO sqlalchemy.engine.base.Engine (194785, 194785, 194785)
2020-03-01 19:30:44,102 INFO sqlalchemy.engine.base.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) no such table: loans
[SQL: INSERT INTO loans (spotreba, bydleni, ostatni) VALUES (?, ?, ?)]
[parameters: (194785, 194785, 194785)]
(Background on this error at: http://sqlalche.me/e/e3q8)

In [8]:
loan1 = Loans(spotreba=194785, bydleni=194785, ostatni=194785)

In [9]:
session.add(loan1)

In [14]:
cols = list(df.columns)

In [15]:
for index, row in df.iterrows():
    
    datum, spotreba, bydleni, ostatni = row[cols[0]],row[cols[1]],row[cols[2]],row[cols[3]]
    loan = Loans(datum=datum, spotreba=spotreba, bydleni=bydleni, ostatni=ostatni)
    session.add(loan)
    print("{}, {}, {}, {} added into the database.".format(datum, spotreba, bydleni, ostatni))

2005-12-31 00:00:00, 182036.4, 282662.2, 58149.7 added into the database.
2006-01-31 00:00:00, 181932.9, 287555.3, 58207.3 added into the database.
2006-02-28 00:00:00, 182050.8, 292363.9, 58634.9 added into the database.
2006-03-31 00:00:00, 183489.5, 298946.3, 59563.1 added into the database.
2006-04-30 00:00:00, 184419.0, 304568.2, 60349.1 added into the database.
2006-05-31 00:00:00, 186758.2, 312268.1, 61220.8 added into the database.
2006-06-30 00:00:00, 193125.3, 321510.3, 62231.5 added into the database.
2006-07-31 00:00:00, 194785.5, 329256.4, 63846.8 added into the database.
2006-08-31 00:00:00, 196423.5, 338140.2, 64232.4 added into the database.
2006-09-30 00:00:00, 196802.8, 346604.9, 68370.3 added into the database.
2006-10-31 00:00:00, 198936.8, 355546.7, 67465.2 added into the database.
2006-11-30 00:00:00, 201561.4, 364463.3, 67813.9 added into the database.
2006-12-31 00:00:00, 210367.5, 375077.3, 69991.4 added into the database.
2007-01-31 00:00:00, 210506.8, 380388.

In [16]:
session.commit()

StatementError: (builtins.TypeError) SQLite Date type only accepts Python date objects as input.
[SQL: INSERT INTO loans (datum, spotreba, bydleni, ostatni) VALUES (?, ?, ?, ?)]
[parameters: [{'bydleni': 194785.5, 'spotreba': 194785.5, 'datum': '2006-04-30', 'ostatni': 194785.5}]]

In [21]:
our_loan = session.query(Loans).filter_by(spotreba=181932.9).first()
our_loan

InvalidRequestError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (sqlite3.OperationalError) no such table: loans
[SQL: INSERT INTO loans (datum, spotreba, bydleni, ostatni) VALUES (?, ?, ?, ?)]
[parameters: ('2005-12-31', 182036.4, 282662.2, 58149.7)]
(Background on this error at: http://sqlalche.me/e/e3q8) (Background on this error at: http://sqlalche.me/e/7s2a)

In [126]:
loan = Loans(datum='2006-04-30', spotreba=194785.5, bydleni=194785.5, ostatni=194785.5)

In [21]:
user1 = User(username='Adam',password='password')

In [22]:
user1.username

'Adam'

In [10]:
ouruser = session.query(User).filter_by(username='Adam')
ouruser

In [12]:
user1 is ouruser

False

In [35]:
user3 = User(username='Tom',password='password')

In [36]:
session.add(user3)

In [37]:
session.commit()

### Querrying

In [38]:
for instance in session.query(User).order_by(User.username):
    print(type(instance))
    print(instance)

<class '__main__.User'>
<User(name='Adam', password='password')>
<class '__main__.User'>
<User(name='Ondra', password='password')>
<class '__main__.User'>
<User(name='Tom', password='password')>


In [41]:
User.query.all()

AttributeError: type object 'User' has no attribute 'query'

### Testing

In [61]:
df = pd.DataFrame([{'c1':10, 'c2':100}, {'c1':11,'c2':110}, {'c1':12,'c2':120}])

In [62]:
df

,c1,c2
0,10,100
1,11,110
2,12,120


In [66]:
for index, row in df.iterrows():
    a,b = row['c1'], row['c2']

In [68]:
b

120

In [75]:
a = df.iterrows()
a

<generator object DataFrame.iterrows at 0x000001CD9FA098B8>

In [74]:
list(a)[0]

(0, c1     10
 c2    100
 Name: 0, dtype: int64)

In [90]:
row = next(df.iterrows())[1]
row

c1     10
c2    100
Name: 0, dtype: int64

In [93]:
for number in row:
    print(number)

10
100


In [28]:
class Postava():
    def __init__(self, rasa, mesto):
        self.rasa = rasa
        self.mesto = mesto
    def __repr__(self):
        return(F"Postava: {self.rasa}, {self.mesto}")

In [24]:
Geralt = Postava('clovek', 'Rivie')

In [25]:
Geralt.__repr__()

'Postava: clovek, Rivie'

In [27]:
print(Geralt)

Postava: clovek, Rivie
